In [1]:
import folium
from folium import Marker, GeoJson
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

In [2]:
root_path="/home/pliu/data_set/kaggle/geospatial/L07"
ais_path=f"{root_path}/AIS_2022_01_01.csv"

In [ ]:
ais=gpd.read_file(ais_path)

In [ ]:
ais.head()